Load libraries.  In this case we need the ndex library, which you can get with "pip install ndex"

In [1]:
#ndex = __import__("ndex")

In [2]:
import ndex2
import requests
import json
import base64

# Connect to NDEx in order to retrieve a network.

In [3]:
anon_ndex = ndex.client.Ndex("http://public.ndexbio.org")

Get the "NCI Pathway Interaction Database - Final Revision - Extended Binary SIF" from NDEx, using its UUID. 

In [3]:
nci_pid_uuid='c0e70804-d848-11e6-86b1-0ac135e8bacf'
nci_pid = ndex2.create_nice_cx_from_server(server='http://public.ndexbio.org', uuid=nci_pid_uuid)

Set the first node in the network to have the attribute "diffusion_input" to 1.0. This is the heat that will be diffused through the network to compute ranks and heats for each node.

In [18]:
node_id, node = next(iter(nci_pid.get_nodes()))
print(node_id)
print("heat source is " + node.get_name())
nci_pid.set_node_attribute(node, "diffusion_input", 1.0)

3431
heat source is CMTM2


Call the Diffusion service

In [19]:
url = 'http://v3.heat-diffusion.cytoscape.io'
payload = nci_pid.to_cx()
headers = {'content-type': 'application/json'}
response = requests.post(url, data=json.dumps(payload), headers=headers)

ConnectionError: ('Connection aborted.', BrokenPipeError(32, 'Broken pipe'))

#The 'data' dictionary in the json response is a dictionary with node IDs as keys and, 'rank' and 'heat' as the values. 

See the top 10 most relevant nodes to your query. 

In [11]:
nos = []
R = ndex.networkn.NdexGraph(cx=response.json()['data'])
for n in R.nodes_iter():
    if R.node[n]['diffusion_output_rank'] < 10:
        R.node[n]['nid'] = n
        nos.append(R.node[n])
nos = sorted(nos, key=lambda k: k['diffusion_output_rank'])
for no in nos:
    print "id: " + str(no['nid']) + " name: " + no['name'] + " rank: " + str(no['diffusion_output_rank']) + " heat: " + str(no['diffusion_output_heat'])


id: 806 name: ITSN2 rank: 0 heat: 0.906123911644
id: 2878 name: BRAF rank: 1 heat: 0.06318075277
id: 2600 name: PAK4 rank: 2 heat: 0.030950992507
id: 2147 name: PPP2R2B rank: 3 heat: 0.0303421980801
id: 2397 name: KSR1 rank: 4 heat: 0.0251069488922
id: 1337 name: RIT1 rank: 5 heat: 0.0222036772476
id: 1340 name: RIT2 rank: 6 heat: 0.0222036772476
id: 1857 name: PPP2R1A rank: 7 heat: 0.0203788459581
id: 1251 name: RANBP10 rank: 8 heat: 0.0198635537603
id: 1431 name: MUC20 rank: 9 heat: 0.0188499154842
